In [68]:
#collection of functions to compute trajectory features
source('featuresCalcsNew.R')

In [70]:
#minimum traj length
cutoffLen<-3
#immobile/mobile threshold
cutoff=2.22236433588659

In [72]:
theFileName<-"/Users/george/Desktop/sRg_calc_csv/GB_165_2022_03_01_HTEndothelial_NonBapta_plate1_2_MMStack_Default_bin10_locsID_tracks_forR.csv"

In [74]:
df<-read.csv(theFileName,header = T)

In [76]:
# the csv file is assumed to have the following columns:
#,track_number,frame,x,y,intensity,id,x [nm],y [nm]
# the header is assumed to be present

In [80]:
#use split to parse frame,x,y by track_number
things<-split(df[,c('frame','x','y')],df$track_number)

In [ ]:
#check track lengths and select those above the length cutoff
#sel has the counting indices
lens<-sapply(things,nrow)
sel<-which(lens>=cutoffLen)
lsel<-length(sel)
cat(paste("done reading",theFileName,lsel,"extracted of length >=",cutoffLen,"\n"))

#takes the indices of the selected tracks and reconstructs the trajectory as an N-by-2 matrix
#where N is the track length = frame_last - frame_first
#missing points will result in NA entires in the trajectory matrix
sRgL<-vector(mode="list",length = lsel)

#loop over the selected track indices
for(i in seq_along(sel)) {
        #"raw" data (a data frame)
      mymat<-things[[sel[i]]]
    #tmat is the trajectory matrix one is as described abov
     tmat<-matrix(nrow=mymat[nrow(mymat),'frame']-mymat[1,'frame']+1,ncol=2)
      tmat[mymat[,1]-mymat[1,1]+1,1]<-mymat[,2]
      tmat[mymat[,1]-mymat[1,1]+1,2]<-mymat[,3]
        #compute sRg for this trajectory
      sRgL[[i]]<-getsRg(tmat)
}
sRg<-unlist(sRgL)

In [82]:
#threshold classification
myclass<-ifelse(sRg<cutoff,'immobile','mobile')

In [84]:
table(myclass)

myclass
immobile   mobile 
    3119     1700 

In [86]:
# Create a data frame with track numbers, sRg values, and classifications
sRg_data <- data.frame(track_number = sel, sRg = sRg, classification = myclass)


In [88]:
# Merge the original data with the new sRg and classification data
merged_df <- merge(df, sRg_data, by.x = "track_number", by.y = "track_number", all.x = TRUE)


In [90]:
# Sort the dataframe to ensure it's in the original order
merged_df <- merged_df[order(merged_df$track_number, merged_df$frame),]


In [92]:
# Create the new filename with '_sRg' tag
new_filename <- sub("\\.csv$", "_sRg.csv", theFileName)


In [94]:
# Save the new CSV file
write.csv(merged_df, file = new_filename, row.names = FALSE)

cat(paste("New file saved as:", new_filename, "\n"))

New file saved as: /Users/george/Desktop/sRg_calc_csv/GB_165_2022_03_01_HTEndothelial_NonBapta_plate1_2_MMStack_Default_bin10_locsID_tracks_forR_sRg.csv 
